# Tune the best ML model for AKGDH

The best = the one with the lowest average MAPE score from "models_filter".

Preprocessing and feature selection is performed after splitting the data (therefore, only the samples in the training data are available for these steps). 

In [1]:
#import libraries
import pandas as pd
import numpy as np
from IPython.display import display

from functools import partial #to pass parameters to function inside another function
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
from sklearn import svm

In [2]:
#import self-made functions
%run /Users/mariekececilia/Documents/master_thesis_code/methods.ipynb

In [3]:
#set a random seed to make reproducible results (used in mutual information)
seed = 0

## Load and clean data

In [4]:
#load transcriptional and fluxomic data
gexp, flux = load_gerosa()

#clean the transcriptional data (remove duplicates)
gexp, groups = clean_gexp_g(gexp)

#define features
X = gexp

#define targets and extract their fluxes
target_labels = ['R_AKGDH']
targets = dict()
for target_label in target_labels:
    targets[target_label] = flux[target_label]

## Create models/pipelines

Choose selection parameters:

In [5]:
cov_filters = [0.2] #same as in the best model
k_numbers = [15, 20, 30] #same k as in the best model + 1 larger and 1 smaller k

Choose learning algorithms:

In [6]:
algorithms = dict({
    'lr': linear_model.LinearRegression(),
    'lasso_a1': linear_model.Lasso(alpha = 1), 
    'lasso_a0.1': linear_model.Lasso(alpha = 0.1),
    'lasso_a0.05': linear_model.Lasso(alpha = 0.05),
    'lasso_a0.001': linear_model.Lasso(alpha = 0.001), 
    'ridge_a1': linear_model.Ridge(alpha = 1),
    'ridge_a0.1': linear_model.Ridge(alpha = 0.1),
    'ridge_a0.05': linear_model.Ridge(alpha = 0.05),
    'ridge_a0.001': linear_model.Ridge(alpha = 0.001),
    })

Make combinations:

In [7]:
def get_pipelines(algorithm, cov_filters, k_numbers):
    models = dict()

    for cov_filter in cov_filters:  
        
        for k in k_numbers:
            #selection by mi – same as in the best model (skipping the other scoring method)
            steps = [
                ('filter', CoVSelector(p = cov_filter)),
                ('selection', SelectKBest(score_func = partial(
                    mutual_info_regression, 
                    random_state=seed), 
                                          k = k)), 
                ('standardize', StandardScaler()),
                ('m', algorithm)
            ]
            models['v%.0f_mi%.0f' % (cov_filter*100, k)] = Pipeline(steps=steps)

    return models

In [8]:
models = dict()

for name, algorithm in algorithms.items():
     models[name]= get_pipelines(algorithm, cov_filters = cov_filters, k_numbers = k_numbers)

## Learning

In [9]:
# evaluate the models and store results in nested dicts: target -> learning algorithm -> scores/preds of all selection combinations
scores = dict()
preds = dict()
for y_name, y in targets.items():
    model_scores = dict()
    model_preds = dict()
    for model_name, pipelines in models.items():
        #get scores
        model_scores[model_name] = compare_models(pipelines, X, y)
        #get predictions
        model_preds[model_name] = get_models_preds(pipelines, X, y)
    scores[y_name] = model_scores
    preds[y_name] = model_preds

/Users/mariekececilia/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.250e-03, tolerance: 3.857e-03
  model = cd_fast.enet_coordinate_descent(
/Users/mariekececilia/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.250e-03, tolerance: 3.857e-03
  model = cd_fast.enet_coordinate_descent(
/Users/mariekececilia/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consi

### Save data to use in other notebooks

Scores and predictions are visualized in the Notebook 'compare_models', which also merges the results with the results from all other models built.

In [10]:
scores_tune_akgdh = scores
preds_tune_akgdh= preds
%store scores_tune_akgdh
%store preds_tune_akgdh

Stored 'scores_tune_akgdh' (dict)
Stored 'preds_tune_akgdh' (dict)


### Scores and predictions

In [11]:
for y in targets.keys():
    for algorithm in algorithms.keys():
        print()
        print(y + '\t(target)') 
        print(algorithm, '\t(learning algorithm)') 
        print('–> summary of all cv split scores for each combination of selection methods:')
        df = scores[y][algorithm][1].sort_values(by = 'average')
        df.index.name = 'Selection combo'
        df.columns.name = 'Score'
        display(df)
        
        print()
        print(y + '\t(target)') 
        print(algorithm + '\t(learning algorithm)') 
        print('–> all individual cv-split scores for each combination of selection methods:')
        df = scores[y][algorithm][0]
        df.columns.name = 'Selection combo'
        display(df)
        
        print()
        print(y + '\t(target)') 
        print(algorithm + '\t(learning algorithm)') 
        print('–> all predictions for each combination of selection methods (+ actual values):')
        df = preds[y][algorithm]
        df.index.name = 'Test set'
        df.columns.name = 'Selection combo'
        display(df)


R_AKGDH	(target)
lr 	(learning algorithm)
–> summary of all cv split scores for each combination of selection methods:


Score,average,std
Selection combo,,
v20_mi20,0.897167,1.588921
v20_mi15,0.971282,1.877622
v20_mi30,1.066749,1.427750



R_AKGDH	(target)
lr	(learning algorithm)
–> all individual cv-split scores for each combination of selection methods:


Selection combo,v20_mi15,v20_mi20,v20_mi30
Test set,,,
Acetate,0.216796,0.000976,0.133150
Fructose,0.321971,0.311837,0.178014
Galactose,0.313537,0.516975,2.388713
Glucose,0.377105,0.461827,0.415111
Glycerol,0.008542,0.019984,0.039246
Gluconate,5.927318,5.073364,4.352453
Pyruvate,0.449941,0.448523,0.440749
Succinate,0.155049,0.343849,0.586556



R_AKGDH	(target)
lr	(learning algorithm)
–> all predictions for each combination of selection methods (+ actual values):


...

Score,average,std
Selection combo,,
v20_mi20,0.897657,1.589341
v20_mi15,0.970504,1.878221
v20_mi30,1.066896,1.428123



R_AKGDH	(target)
ridge_a0.001	(learning algorithm)
–> all individual cv-split scores for each combination of selection methods:


Selection combo,v20_mi15,v20_mi20,v20_mi30
Test set,,,
Acetate,0.215808,0.000879,0.133178
Fructose,0.320838,0.311529,0.178006
Galactose,0.309024,0.521333,2.389587
Glucose,0.376233,0.460792,0.414851
Glycerol,0.008533,0.019843,0.039308
Gluconate,5.928199,5.074797,4.353316
Pyruvate,0.449869,0.448534,0.440749
Succinate,0.155529,0.343549,0.586173



R_AKGDH	(target)
ridge_a0.001	(learning algorithm)
–> all predictions for each combination of selection methods (+ actual values):


Selection combo,actual,v20_mi15,v20_mi20,v20_mi30
Test set,,,,
Acetate,4.267163,3.346276,4.270915,3.698871
Fructose,3.874895,5.118110,5.082036,4.564651
Galactose,0.236073,0.163120,0.359145,0.800188
Glucose,2.138073,1.333659,1.152867,1.251092
Glycerol,1.840459,1.856163,1.876980,1.768114
Gluconate,0.182358,1.263415,1.107790,0.976221
Pyruvate,7.415655,4.079583,4.089479,4.147212
Succinate,2.351983,1.986182,1.543961,0.973313
